# Installing the Library

In [1]:
!pip install -q groq
!pip install -q pymupdf
!pip install -q pillow
!pip install -q torch
!pip install -q torchvision
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q faiss-cpu
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 83.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu

# Importing necessary libraries

In [2]:
import os
import shutil
import fitz  # PyMuPDF for reading PDFs
from PIL import Image  # for image processing
import numpy as np

from groq import Groq  # Groq LLM client
from transformers import AutoModel, AutoTokenizer  # For MiniCPM VLM model
from sentence_transformers import SentenceTransformer  # Sentence embeddings
import faiss  # For semantic search

2025-04-13 10:49:33.585945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744541373.778906      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744541373.829907      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Initialize Groq LLM Client with your API key

In [3]:
api_key = "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"  # 🔐 Replace with your Groq key
client = Groq(api_key=api_key)

In [4]:
# Store chat history for coherent conversations
history = []

# Load the Vision-Language Model for OCR from images

In [5]:
vlm_model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
vlm_tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
vlm_model.eval()  # Set model to evaluation mode

config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5-int4:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

resampler.py:   0%|          | 0.00/5.49k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((4096,),


# Load a lightweight sentence embedding model

In [6]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Function to perform general chat without PDF using Groq LLM

In [7]:
def ask_groq(user_message):
    history.append({"role": "user", "content": user_message})  # Add user's message
    response = client.chat.completions.create(  # Get LLM response
        model="llama3-8b-8192",
        messages=history,
        temperature=0.7,
        max_tokens=512,
    )
    reply = response.choices[0].message.content  # Extract reply
    history.append({"role": "assistant", "content": reply})  # Save reply in history
    return reply

# Convert PDF pages into image files

In [8]:
def pdf_to_images(pdf_path, output_folder="output_images"):
    pdf_document = fitz.open(pdf_path)  # Open PDF
    if os.path.exists(output_folder):  # Clean output folder if exists
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)  # Create fresh output folder

    for i in range(len(pdf_document)):  # Loop through pages
        page = pdf_document.load_page(i)  # Load each page
        pix = page.get_pixmap()  # Convert to pixel map
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)  # Convert to PIL Image
        img.save(os.path.join(output_folder, f"page_{i + 1}.png"))  # Save image

    return len(pdf_document), output_folder  # Return total pages and path


# Extract text from image using MiniCPM model

In [9]:
def handwriting_ocr_img(image_path):
    img = Image.open(image_path).convert('RGB')  # Open and convert image
    question = 'Extract the text from the image. just give only image text'
    msgs = [{'role': 'user', 'content': question}]
    system_role = 'You are an AI that extracts and transcribes text from images accurately.'

    # Use MiniCPM to process image and extract text
    res = vlm_model.chat(
        image=img,
        msgs=msgs,
        tokenizer=vlm_tokenizer,
        sampling=True,
        temperature=0.7,
        system_prompt=system_role
    )

    return ''.join(res)  # Return extracted text

# Function to extract and combine all text from PDF

In [10]:
def extract_text_from_pdf(pdf_path):
    _, image_dir = pdf_to_images(pdf_path)  # Convert PDF to images
    full_text = ""
    for img in sorted(os.listdir(image_dir)):  # Loop over each image
        path = os.path.join(image_dir, img)
        full_text += handwriting_ocr_img(path)  # Add extracted text
    return full_text

# Split large text into smaller chunks for embedding

In [11]:
def split_text_into_chunks(text, chunk_size=500):
    sentences = text.split('. ')  # Split into sentences
    chunks, current = [], ""  # Init chunk list
    for sentence in sentences:
        if len(current) + len(sentence) < chunk_size:
            current += sentence + ". "  # Add to current chunk
        else:
            chunks.append(current.strip())  # Save current chunk
            current = sentence + ". "
    if current:
        chunks.append(current.strip())  # Add remaining chunk
    return chunks

# Build FAISS index from embedded chunks

In [12]:
def build_faiss_index(text_chunks):
    embeddings = embedding_model.encode(text_chunks)  # Embed chunks
    dim = embeddings.shape[1]  # Dimensionality
    index = faiss.IndexFlatL2(dim)  # Create FAISS index
    index.add(np.array(embeddings))  # Add embeddings
    return index, embeddings

# Search relevant chunks and ask Groq to generate answer

In [13]:
def search_faiss_and_ask(question, chunks, index):
    q_embed = embedding_model.encode([question])  # Embed question
    _, top_indices = index.search(np.array(q_embed), k=3)  # Search top 3
    retrieved = "\n".join([chunks[i] for i in top_indices[0]])  # Get top chunks

    # Format prompt with retrieved context
    context = f"Answer the question based on the following:\n{retrieved}"
    messages = [
        {"role": "system", "content": "You are a helpful AI that answers based on given context."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ]

    # Ask Groq to answer using retrieved context
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        temperature=0.7,
        max_tokens=500,
    )
    return response.choices[0].message.content



# Function to summarize entire document

In [14]:
def summarize_large_legal_text(text):
    from textwrap import dedent  # For formatting
    chunk_size = 8000
    all_summaries = []

    # Break large text into chunks
    for i in range(0, len(text), chunk_size):
        sub_text = text[i:i + chunk_size]
        prompt = dedent(f"""
        You are a legal AI assistant tasked with summarizing a legal document provided as text. Please perform the following:

        1. Summarize the entire document concisely in 3-5 sentences.
        2. Extract important dates.
        3. Identify individuals and organizations mentioned.

        TEXT TO ANALYZE:
        {sub_text}
        """)

        # Get Groq summary for each chunk
        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500,
        )

        all_summaries.append(response.choices[0].message.content)

    return "\n\n--- Summary Chunk ---\n\n".join(all_summaries)  # Combine summaries


# ========== MAIN ENTRY POINT ==========

In [15]:
def main():
    print("\n🧠 Welcome to the AI Legal Chatbot 🧠")
    while True:
        # Show user menu
        print("\nChoose an option:")
        print("1. Chat without PDF")
        print("2. Chat with PDF")
        print("3. Exit")

        choice = input("Enter your choice (1/2/3): ")

        # Match-case switch (Python 3.10+)
        match choice:
            case "1":
                while True:
                    user_input = input("\nYou: ")  # Get user input
                    if user_input.lower() in ["exit", "quit"]:
                        break
                    response = ask_groq(user_input)  # Get response from Groq
                    print("AI:", response)

            case "2":
                pdf_path = input("\nEnter path to your legal PDF: ").strip()
                if not os.path.isfile(pdf_path):  # Check if file exists
                    print("Invalid file path. Please try again.")
                    continue

                print("Processing document...")
                text = extract_text_from_pdf(pdf_path)  # Extract OCR text
                chunks = split_text_into_chunks(text)  # Chunk the text
                index, _ = build_faiss_index(chunks)  # Build search index

                while True:
                    # Show PDF submenu
                    print("\nOptions:")
                    print("a. Ask question from document")
                    print("b. Summarize document")
                    print("c. Back to main menu")
                    sub_choice = input("Enter your choice (a/b/c): ")

                    if sub_choice == 'a':
                        q = input("Your question: ")
                        answer = search_faiss_and_ask(q, chunks, index)
                        print("Answer:", answer)

                    elif sub_choice == 'b':
                        summary = summarize_large_legal_text(text)
                        print(summary)

                    elif sub_choice == 'c':
                        break  # Go back to main menu

                    else:
                        print("Invalid option. Try again.")

            case "3":
                print("👋 Exiting. Thank you!")  # Exit program
                break

            case _:
                print("Invalid choice. Please enter 1, 2, or 3.")  # Handle invalid input

# Run program only when executed directly
if __name__ == "__main__":
    main()


🧠 Welcome to the AI Legal Chatbot 🧠

Choose an option:
1. Chat without PDF
2. Chat with PDF
3. Exit


Enter your choice (1/2/3):  1

You:  hello


AI: Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?



You:  let me about the indian law


AI: Indian law, also known as the law of India, refers to the legal system and laws of the Republic of India. The Indian legal system is based on the common law system and the Indian Constitution, which was adopted on January 26, 1950. Here are some key aspects of Indian law:

**Constitution**: The Indian Constitution is the supreme law of the land and establishes the framework of the Indian government. It consists of 395 articles and 12 schedules.

**Sources of Law**: Indian law derives from the following sources:

1. The Constitution
2. Statutes (Acts of Parliament)
3. Common Law (judge-made law)
4. Customary Law (traditional practices and customs)
5. International Law (treaties and conventions)

**Types of Laws**: Indian law encompasses various types of laws, including:

1. **Criminal Law**: Deals with crimes and punishments, such as the Indian Penal Code.
2. **Civil Law**: Deals with disputes between individuals, such as contract law and tort law.
3. **Family Law**: Deals with fami

...


Choose an option:
1. Chat without PDF
2. Chat with PDF
3. Exit


Enter your choice (1/2/3):  2

Enter path to your legal PDF:  exit


Invalid file path. Please try again.

Choose an option:
1. Chat without PDF
2. Chat with PDF
3. Exit


Enter your choice (1/2/3):  3


👋 Exiting. Thank you!
